In [9]:
#This Code predict Label for SVF Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from keras.optimizers import Adam
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_02112019.csv')
# frac=1 means return all rows (in random order)
data = data.sample(frac=1)

y = list(data['Label']) 
x = list(data['Title_Description']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
adam = Adam(lr=0.04)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=10, batch_size=40)
    model.save_weights('./Roslyn_TD_Label_Predict_Single.h5')
    predicts = model.predict(x_test, batch_size=40)  

y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Epoch 1/10
8762/8762 [==============================] - 611s 70ms/step - loss: 1.9313 - acc: 0.4388
Epoch 2/10
8762/8762 [==============================] - 603s 69ms/step - loss: 1.6235 - acc: 0.4912
Epoch 3/10
8762/8762 [==============================] - 600s 68ms/step - loss: 1.5699 - acc: 0.5007
Epoch 4/10
8762/8762 [==============================] - 604s 69ms/step - loss: 1.5698 - acc: 0.5090
Epoch 5/10
8762/8762 [==============================] - 599s 68ms/step - loss: 1.5223 - acc: 0.5175
Epoch 6/10
8762/8762 [==============================] - 599s 68ms/step - loss: 1.4985 - acc: 0.5226
Epoch 7/10
8762/8762 [==============================] - 606s 69ms/step - loss: 1.4843 - acc: 0.5334
Epoch 8/10
8762/8762 [==============================] - 599s 68ms/step - loss: 1.4702 - acc: 0.5305
Epoch 9/10
8762/8762 [==============================] - 598s 68ms/step - loss: 1.4624 - acc: 0.5347
Epoch 10/10
8762/8762 [==============================] - 599s 68ms/step - loss: 1.4602 - acc: 0.5381

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# New Section